In [ ]:
import matplotlib as mpl
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go

![](Business_understanding.jpeg)

# Data Load

In [ ]:
df_analyse=pd.read_csv('E:/ads_covid-19/data/processed/COVID_small_sync_timeline_table.csv',sep=';')

In [ ]:
df_analyse.sort_values('date',ascending=True).head()

In [ ]:
country_list = df_analyse.columns[1:]

# Helper Functions

In [ ]:
def quick_plot(x_in, df, y_scale = 'log', slider = False, xname = ' ', yname = ' ', figname = ' '):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
        xname : str
            Title of the X axis
        yname : str
            Title of the Y axis
        figname : str
            Title of the figure
        
        Returns:
        ----------
        Return type: Figure
    
    """    
    fig = go.Figure()
    for each in df.columns:
        fig.add_trace(go.Scatter(x=x_in, 
                                 y=df[each], 
                                 name= each,
                                 mode= 'markers+lines',
                                 line_width = 1,
                                 marker_size = 3)
                     )
    fig.update_layout(xaxis_title = xname,
                      yaxis_title = yname,
                      width = 1000, 
                      height = 800,
                      title={
                        'text': figname,
                        'y':0.9,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'}
                    )
    fig.update_yaxes(type = y_scale)
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )    
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Fitting a Polynomial Curve

- Polynomial interpolation example: https://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html#sphx-glr-auto-examples-linear-model-plot-polynomial-interpolation-py

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [ ]:
df_poly_check = df_analyse.iloc[0:27,4:-1].reset_index()

In [ ]:
df_poly_check.head()

In [ ]:
y = df_poly_check[['India','US','Italy','Spain','Germany']].unstack().sort_index(axis=0, level=1)
y.tail()

In [ ]:
# Prepare train and test data
test_points = 28
y_train = y[0:-test_points-1]
y_test = y[-test_points:]
X_train = np.arange(len(y_train)).reshape(-1,1)/5.0
X_test = np.arange(len(y_train),len(y_train)+test_points).reshape(-1,1)/5.0

In [ ]:
import seaborn
seaborn.set(rc={'figure.figsize':(16,8)})  # plot formatting, prepare for subplot 

- PolynomialFeatures: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html
- make_pipeline: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html?highlight=pipeline#sklearn.pipeline.make_pipeline

In [ ]:
def PolynomialRegression(degree = 2, **kwarg):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwarg))

In [ ]:
fig, ax = plt.subplots(1,1)
ax.scatter(np.arange(len(y))/5, y, color = 'blue') 
ax.axvspan((len(y)-test_points-1)/5, len(y)/5, facecolor='r', alpha=0.5)
ax.set_xlabel("Day")
ax.set_ylabel("Number of cases")

for degree in [1,3,7,15]:
    y_hat_insample = PolynomialRegression(degree).fit(X_train, y_train).predict(X_train)
    y_hat_test = PolynomialRegression(degree).fit(X_train, y_train).predict(X_test)
    X_plot=np.concatenate((X_train, X_test), axis=None)
    y_plot=np.concatenate((y_hat_insample, y_hat_test), axis=None)
    
    ax.plot(X_plot, y_plot, label='degree={0}'.format(degree)+ 
                 '     MAPE train:  ' + str(mean_absolute_percentage_error(y_hat_insample, y_train))[0:3]
                 +'    MAPE test    ' +str(mean_absolute_percentage_error(y_hat_test, y_test))[0:3])
ax.set_ylim(100, 150000)
ax.set_yscale('log')
ax.legend(loc='best',
           prop={'size': 16});